In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error


path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-13 08:39:55,913 utils 400 [INFO]    [logger_func] start 


In [71]:
#========================================================================
# CV CHECKER 
# Data Load
def get_cv_score(model_path):
    base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
    train = base[~base[target].isnull()]

    tmp = utils.read_pkl_gzip(model_path)
    if 'pred_mean' in tmp.columns:
        pred_col = 'pred_mean'
    elif 'prediction' in tmp.columns:
        pred_col = 'prediction'
    tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
    
    train['pred'] = tmp[pred_col]

    y_train = train[target].values
    y_pred = train['pred'].values
    score = np.sqrt(mean_squared_error(y_train, y_pred))
    print(f"CV: {score}")
    return score
#========================================================================

In [ ]:
#========================================================================
# 1:N Corr Checker
from itertools import combinations
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz').set_index(key).query('no_out_flg==1')
ens_list = glob.glob('../stack/*.gz')

base_path = '../ensemble/0212_105_all_lgb_out_partall_row201917_lr0.01_235feats_10seed_57leaves_iter1428_OUT0_CV3-6206463759490277_LB.gz'
base_model = utils.read_pkl_gzip(base_path)[[key, 'pred_mean']].set_index(key)
base['base_pred'] = base_model['pred_mean']
cv1 = re.search(r'CV([^/.]*)_LB.gz', base_path).group(1)
base_len = len(base)

#=======================================================================
# Classifier
# clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
# clf['pred_mean_2'] = clf_2['pred_mean']
# clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# base['clf_pred'] = clf['clf_pred']
# utils.to_pkl_gzip(obj=base.reset_index(), path='../input/base_clf')
# sys.exit()
#========================================================================

#========================================================================
# 相関

def get_corr_ensemble(path, base):
    '''
    1. ベースモデルとの相関を個別に取得する
    2. ベースモデルにpath_listのモデルの予測をjoinし、相関行列を取得する
    '''
    
    if key in base.columns:
        base = base.set_index(key)
    
    #========================================================================
    # 個別に相関を見ていく場合
    if str(type(path)).count('str'):
    
        if path.count('CV1'):
            return 0, ''
        if path.count('binary'):
            return 0, ''
        tmp = utils.read_pkl_gzip(path)
    
        try:
            tmp = tmp.reset_index()[[key, 'pred_mean']].set_index(key)
            base['tmp_pred'] = tmp.reset_index()['pred_mean']
        except KeyError:
            tmp = tmp.reset_index()[[key, 'prediction']].set_index(key)
            base['tmp_pred'] = tmp['prediction']
    
        if path.count('LB'):
            try:
                cv2 = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
            except AttributeError:
                return 0, ''
        else:
            cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
    
        corr = np.corrcoef(base['base_pred'], base['tmp_pred'].values).min()
    
        if corr>0 and corr<0.98:
            logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
            
            return corr, path
        else:
            return 0, ''
    #========================================================================
    
    #========================================================================
    # まとめて相関行列を見たい場合
    elif str(type(path)).count('list'):
        path_list = path
        for path in path_list:
        
            if path.count('CV1'):
                return 0, ''
            if path.count('binary'):
                return 0, ''
            tmp = utils.read_pkl_gzip(path)

            if 'pred_mean' in tmp.columns:
                pred_col = 'pred_mean'
            elif 'prediction' in tmp.columns:
                pred_col = 'prediction'
                
            tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
                
            if path.count('LB'):
                try:
                    cv2 = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
                except AttributeError:
                    return 0, ''
            else:
                cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
                
            base[f'pr_{cv2[:6]}'] = tmp[pred_col]
            
        drop_cols = [col for col in base.columns if col.count('term') or col.count('no_out_flg') or col in ignore_list or col.count('clf')]
        base.drop(drop_cols, axis=1, inplace=True)
        base.sort_index(axis=1, inplace=True)
        mx_corr = base.corr(method='pearson')
#         mx_cor = mx_corr.drop(drop_cols, axis=1).loc[~mx_corr.index.isin(drop_cols)]
        
        return mx_corr
    #========================================================================
    

corr_list = []
path_list = [] 
for path in ens_list:
    corr, path = get_corr_ensemble(path, base)
    if corr>0:
        corr_list.append(corr)
        path_list.append(path)
#========================================================================

In [ ]:
# top_ens_list = path_list.copy()
df_corr = pd.DataFrame([path_list, corr_list]).T
df_corr.columns = ['path', 'corr']
df_corr.sort_values(by='corr', ascending=False, inplace=True)
df_corr['cv'] = df_corr['path'].map(lambda x: re.search(r'CV([^/.]*)_LB.gz', x.replace('.', '-')).group(1))
df_corr.reset_index(inplace=True, drop=True)
df_corr

In [103]:
import shutil
move_path = df_corr[df_corr['cv'].map(lambda x: str(x)[:6])=='3-7362']['path'].values[0]
shutil.move(move_path, '../ensemble/lgb_ensemble/')

'../ensemble/lgb_ensemble/0213_000_all_lgb_obj-regression_out_part-all_valid-term_ESET0_row201917_lr1.0_235feats_1seed_70leaves_iter1_OUT0_CV3-7362871662281507_LB.gz'

In [64]:
top_corr = get_corr_ensemble(path=top_ens_list, base=base)
display(top_corr)
top_corr.to_csv('../output/0213_elo_corr_ensemble_heatmap.csv', index=True)

In [109]:
model_path = '../stack/0209_164_stack_lgb_lr0.01_338feats_1seed_70leaves_iter995_OUT29.9569_CV3-6297296108420443_LB.gz'
NN_path = '../ensemble/NN_ensemble/*.gz'
nn_path_list = glob.glob(NN_path)
# for model_path in nn_path_list:
#     get_cv_score(model_path)
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')
nn_corr = get_corr_ensemble(path=nn_path_list, base=base)
nn_corr

,pr_3-7753,pr_3-7798,pr_3-7806,pr_3-7807,pr_3-7827,pr_3-7852,pr_3-7855,pr_3-7884
pr_3-7753,1.000000,0.872396,0.851467,0.778420,0.820193,0.844610,0.804543,0.819744
pr_3-7798,0.872396,1.000000,0.864499,0.799611,0.831859,0.851006,0.822094,0.826828
pr_3-7806,0.851467,0.864499,1.000000,0.818127,0.843877,0.906750,0.836875,0.846349
pr_3-7807,0.778420,0.799611,0.818127,1.000000,0.816613,0.803077,0.809814,0.840605
pr_3-7827,0.820193,0.831859,0.843877,0.816613,1.000000,0.838519,0.849409,0.850854
pr_3-7852,0.844610,0.851006,0.906750,0.803077,0.838519,1.000000,0.828511,0.841070
pr_3-7855,0.804543,0.822094,0.836875,0.809814,0.849409,0.828511,1.000000,0.837329
pr_3-7884,0.819744,0.826828,0.846349,0.840605,0.850854,0.841070,0.837329,1.000000
